# 🌌 T-FAN Topology Screensaver - WebGL Version

**Living visualization of topological data analysis, hyperbolic geometry, and multi-objective optimization**

This notebook runs the T-FAN topology screensaver directly in your browser using Three.js and WebGL.

## 🎨 Four Visualization Modes

1. **Barcode Nebula** 🌠 - 3D persistence barcodes floating in space
2. **Landscape Waterfall** 🌊 - Flowing persistence landscapes
3. **Poincaré Orbits** 🪐 - Hyperbolic embeddings on disk
4. **Pareto Galaxy** ⭐ - Multi-objective configuration stars

## 🚀 Quick Start

1. Run the setup cell (installs nothing - pure web!)
2. Run the screensaver cell
3. Use controls to cycle modes, adjust settings

**Note:** Works best in Chrome/Firefox. No GPU needed!

In [ ]:
# Setup - No dependencies needed!
# The screensaver uses Three.js from CDN

print("✅ Setup complete!")
print("\n🎨 Ready to launch T-FAN Topology Screensaver")
print("\nRun the next cell to start the visualization.")

In [ ]:
#@title 🌌 Launch Screensaver { run: "auto" }
#@markdown Choose your visualization mode and settings:

mode = "landscape" #@param ["barcode", "landscape", "poincare", "pareto"]
particle_count = 800 #@param {type:"slider", min:200, max:2000, step:100}
auto_rotate = True #@param {type:"boolean"}
show_metrics = True #@param {type:"boolean"}
animation_speed = 1.0 #@param {type:"slider", min:0.1, max:2.0, step:0.1}

from IPython.display import HTML
import json

# Generate screensaver HTML
html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <style>
        body {{
            margin: 0;
            overflow: hidden;
            background: #020306;
            font-family: monospace;
        }}
        #container {{
            width: 100%;
            height: 600px;
            position: relative;
        }}
        #hud {{
            position: absolute;
            top: 10px;
            left: 10px;
            color: white;
            background: rgba(0, 0, 0, 0.7);
            padding: 10px;
            border-radius: 5px;
            font-size: 12px;
            z-index: 100;
        }}
        #controls {{
            position: absolute;
            bottom: 10px;
            left: 50%;
            transform: translateX(-50%);
            background: rgba(102, 126, 234, 0.9);
            padding: 10px 20px;
            border-radius: 20px;
            color: white;
            font-weight: bold;
            z-index: 100;
        }}
        .btn {{
            background: rgba(255, 255, 255, 0.2);
            border: 1px solid white;
            color: white;
            padding: 5px 15px;
            margin: 0 5px;
            border-radius: 5px;
            cursor: pointer;
            font-family: monospace;
        }}
        .btn:hover {{
            background: rgba(255, 255, 255, 0.3);
        }}
    </style>
</head>
<body>
    <div id="container">
        <canvas id="canvas"></canvas>
        {'<div id="hud"><div id="mode">Mode: ' + mode.upper() + '</div><div id="fps">FPS: 60</div></div>' if show_metrics else ''}
        <div id="controls">
            <button class="btn" onclick="cycleMode()">Cycle Mode (M)</button>
            <button class="btn" onclick="togglePause()">Pause (P)</button>
            <button class="btn" onclick="resetCamera()">Reset Camera</button>
        </div>
    </div>

    <script src="https://cdnjs.cloudflare.com/ajax/libs/three.js/r160/three.min.js"></script>
    <script src="https://cdn.jsdelivr.net/npm/three@0.160.0/examples/js/controls/OrbitControls.js"></script>

    <script>
        const MODES = ['barcode', 'landscape', 'poincare', 'pareto'];
        let currentMode = MODES.indexOf('{mode}');
        let paused = false;
        let time = 0;
        const PARTICLE_COUNT = {particle_count};
        const SPEED = {animation_speed};
        
        // Setup Three.js
        const canvas = document.getElementById('canvas');
        const scene = new THREE.Scene();
        scene.background = new THREE.Color(0x020306);
        
        const camera = new THREE.PerspectiveCamera(60, window.innerWidth / 600, 0.1, 100);
        camera.position.set(0, 5, 10);
        
        const renderer = new THREE.WebGLRenderer({{ canvas, antialias: true }});
        renderer.setSize(window.innerWidth, 600);
        
        const controls = new THREE.OrbitControls(camera, canvas);
        controls.enableDamping = true;
        controls.autoRotate = {str(auto_rotate).lower()};
        controls.autoRotateSpeed = 0.5;
        
        // Lights
        scene.add(new THREE.AmbientLight(0x404040, 1.0));
        const light1 = new THREE.PointLight(0x667eea, 2.0);
        light1.position.set(10, 10, 10);
        scene.add(light1);
        
        // Mode implementations (simplified for Colab)
        function createBarcode() {{
            clearScene();
            for (let i = 0; i < 150; i++) {{
                const geometry = new THREE.BoxGeometry(0.05, 0.05, Math.random() * 2);
                const material = new THREE.MeshPhongMaterial({{
                    color: new THREE.Color().setHSL(0.6 + Math.random() * 0.2, 0.8, 0.6)
                }});
                const mesh = new THREE.Mesh(geometry, material);
                mesh.position.set(
                    Math.random() * 6 - 3,
                    (i / 150) * 10 - 5,
                    Math.random() * 4 - 2
                );
                mesh.userData.type = 'barcode';
                mesh.userData.initialY = mesh.position.y;
                scene.add(mesh);
            }}
        }}
        
        function createLandscape() {{
            clearScene();
            for (let layer = 0; layer < 6; layer++) {{
                const vertices = [];
                const colors = [];
                for (let i = 0; i < 100; i++) {{
                    const x = (i / 100) * 10 - 5;
                    const y = Math.sin(i * 0.2 + layer) * (6 - layer) / 6;
                    const z = layer * 1.5 - 4;
                    vertices.push(x, y, z);
                    const color = new THREE.Color().setHSL(0.5 + layer / 12, 0.8, 0.5);
                    colors.push(color.r, color.g, color.b);
                }}
                const geometry = new THREE.BufferGeometry();
                geometry.setAttribute('position', new THREE.Float32BufferAttribute(vertices, 3));
                geometry.setAttribute('color', new THREE.Float32BufferAttribute(colors, 3));
                const line = new THREE.Line(geometry, new THREE.LineBasicMaterial({{ vertexColors: true, linewidth: 2 }}));
                line.userData.type = 'landscape';
                scene.add(line);
            }}
        }}
        
        function createPoincare() {{
            clearScene();
            const circleGeometry = new THREE.RingGeometry(4.9, 5.0, 128);
            const circleMaterial = new THREE.MeshBasicMaterial({{ color: 0x667eea, side: THREE.DoubleSide, transparent: true, opacity: 0.5 }});
            const circle = new THREE.Mesh(circleGeometry, circleMaterial);
            circle.rotation.x = Math.PI / 2;
            scene.add(circle);
            
            const positions = [];
            const colors = [];
            for (let i = 0; i < PARTICLE_COUNT; i++) {{
                const r = Math.pow(Math.random(), 0.9) * 4.5;
                const theta = Math.random() * Math.PI * 2;
                positions.push(r * Math.cos(theta), (Math.random() - 0.5) * 0.5, r * Math.sin(theta));
                const color = new THREE.Color().setHSL(0.5 + r / 10, 0.8, 0.6);
                colors.push(color.r, color.g, color.b);
            }}
            const geometry = new THREE.BufferGeometry();
            geometry.setAttribute('position', new THREE.Float32BufferAttribute(positions, 3));
            geometry.setAttribute('color', new THREE.Float32BufferAttribute(colors, 3));
            const points = new THREE.Points(geometry, new THREE.PointsMaterial({{ size: 0.1, vertexColors: true }}));
            points.userData.type = 'poincare';
            scene.add(points);
        }}
        
        function createPareto() {{
            clearScene();
            const positions = [];
            const colors = [];
            for (let i = 0; i < 220; i++) {{
                positions.push(
                    (Math.random() - 0.5) * 10,
                    (Math.random() - 0.5) * 2,
                    (Math.random() - 0.5) * 10
                );
                const color = new THREE.Color().setHSL(0.5 + Math.random() * 0.3, 0.9, 0.6);
                colors.push(color.r, color.g, color.b);
            }}
            const geometry = new THREE.BufferGeometry();
            geometry.setAttribute('position', new THREE.Float32BufferAttribute(positions, 3));
            geometry.setAttribute('color', new THREE.Float32BufferAttribute(colors, 3));
            const points = new THREE.Points(geometry, new THREE.PointsMaterial({{ 
                size: 0.2, 
                vertexColors: true, 
                transparent: true, 
                blending: THREE.AdditiveBlending 
            }}));
            points.userData.type = 'pareto';
            scene.add(points);
        }}
        
        function clearScene() {{
            scene.children = scene.children.filter(child => 
                child instanceof THREE.AmbientLight || 
                child instanceof THREE.PointLight
            );
        }}
        
        function cycleMode() {{
            currentMode = (currentMode + 1) % MODES.length;
            const mode = MODES[currentMode];
            if (mode === 'barcode') createBarcode();
            else if (mode === 'landscape') createLandscape();
            else if (mode === 'poincare') createPoincare();
            else if (mode === 'pareto') createPareto();
            
            const modeEl = document.getElementById('mode');
            if (modeEl) modeEl.textContent = 'Mode: ' + mode.toUpperCase();
        }}
        
        function togglePause() {{
            paused = !paused;
        }}
        
        function resetCamera() {{
            camera.position.set(0, 5, 10);
            controls.target.set(0, 0, 0);
            controls.update();
        }}
        
        // Keyboard controls
        document.addEventListener('keydown', (e) => {{
            if (e.key === 'm' || e.key === 'M') cycleMode();
            else if (e.key === 'p' || e.key === 'P') togglePause();
            else if (e.key === 'r' || e.key === 'R') resetCamera();
        }});
        
        // Animation loop
        let lastTime = Date.now();
        let fps = 60;
        
        function animate() {{
            requestAnimationFrame(animate);
            
            const now = Date.now();
            const delta = (now - lastTime) / 1000;
            lastTime = now;
            fps = Math.round(1 / delta);
            
            if (!paused) {{
                time += delta * SPEED;
                
                scene.children.forEach(child => {{
                    if (child.userData.type === 'barcode') {{
                        child.position.y = child.userData.initialY + Math.sin(time + child.position.x) * 0.2;
                    }} else if (child.userData.type === 'poincare') {{
                        child.rotation.y += 0.001;
                    }} else if (child.userData.type === 'pareto') {{
                        child.rotation.y += 0.002;
                    }}
                }});
            }}
            
            controls.update();
            renderer.render(scene, camera);
            
            const fpsEl = document.getElementById('fps');
            if (fpsEl) fpsEl.textContent = 'FPS: ' + fps;
        }}
        
        // Initialize
        cycleMode();
        currentMode = MODES.indexOf('{mode}');
        if (MODES[currentMode] === 'barcode') createBarcode();
        else if (MODES[currentMode] === 'landscape') createLandscape();
        else if (MODES[currentMode] === 'poincare') createPoincare();
        else if (MODES[currentMode] === 'pareto') createPareto();
        
        animate();
        
        // Handle window resize
        window.addEventListener('resize', () => {{
            camera.aspect = window.innerWidth / 600;
            camera.updateProjectionMatrix();
            renderer.setSize(window.innerWidth, 600);
        }});
    </script>
</body>
</html>
"""

display(HTML(html_content))

print("\n🌌 Screensaver is running!")
print("\n⌨️  Keyboard Controls:")
print("  M - Cycle through modes")
print("  P - Pause/unpause")
print("  R - Reset camera")
print("\n🖱️  Mouse Controls:")
print("  Left drag - Rotate")
print("  Right drag - Pan")
print("  Scroll - Zoom")

## 📐 Mathematical Foundations

### Barcode Nebula
- **Persistence Diagrams**: Birth-death pairs of topological features
- **Vietoris-Rips Filtration**: Build simplicial complexes at increasing scales
- **Bar Length**: Persistence = robustness to noise

### Landscape Waterfall
- **Persistence Landscapes**: Functional summaries λ₁(t) ≥ λ₂(t) ≥ ... ≥ λₖ(t)
- **Statistical Analysis**: Enable averaging and hypothesis testing
- **Stability**: Small changes in data → small changes in landscape

### Poincaré Orbits
- **Hyperbolic Geometry**: Constant negative curvature
- **Poincaré Disk**: {z ∈ ℂ : |z| < 1} with hyperbolic metric
- **Hierarchy Embedding**: Distance from center = hierarchy level

### Pareto Galaxy
- **Pareto Optimality**: No point strictly dominates another
- **Multi-Objective**: Balance accuracy, latency, EPR-CV, topology gap, energy
- **Hypervolume**: Quality indicator for Pareto fronts

## 🔬 References

- Edelsbrunner & Harer (2010), *Computational Topology*
- Bubenik (2015), *Statistical Topological Data Analysis using Persistence Landscapes*
- Nickel & Kiela (2017), *Poincaré Embeddings for Learning Hierarchical Representations*
- Miettinen (1998), *Nonlinear Multiobjective Optimization*

## 🚀 Next Steps

- Adjust settings in the form above and re-run
- Try different modes and particle counts
- Experiment with animation speeds
- Use keyboard/mouse controls for interaction

## 📚 More Resources

- [T-FAN GitHub Repository](https://github.com/theadamsfamily1981-max/Quanta-meis-nib-cis)
- [Python/VisPy Version](https://github.com/theadamsfamily1981-max/Quanta-meis-nib-cis/tree/main/tfan/viz/screensaver)
- [WebGL Version](https://github.com/theadamsfamily1981-max/Quanta-meis-nib-cis/tree/main/tfan/viz/screensaver/web)

---

**Built with Three.js, WebGL, and mathematical beauty** ✨